In [32]:
from flask import Flask, render_template, request
from pandas import read_csv
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

# Read dataset from the csv file
filename = "E:\\Work\\FinalProjectData\\unsupervised_data.csv"
dataset  = read_csv(filename)

dataset.pop('name')
dataset.pop('phoneNumber')
dataset.pop('address')

le = LabelEncoder()
for col in dataset.columns:
    if col == 'city':
        le.fit(dataset[col])
        dataset[col] = le.transform(dataset[col])
        
features = ['city', 'age', 'calls', 'activeCalls', 'sms', 'data', 'subscription_a', 
            'subscription_b', 'subscription_c', 'subscription_d']

X = dataset.loc[:, features].values
y = dataset.loc[:, ['plan']].values

kmeans = KMeans(n_clusters=7, random_state=0).fit(X)

def file_process(file, n_cluster=7):
    dataset  = read_csv(file)

    dataset.pop('name')
    dataset.pop('phoneNumber')
    dataset.pop('address')

    le = LabelEncoder()
    for col in dataset.columns:
        if col == 'city':
            le.fit(dataset[col])
            dataset[col] = le.transform(dataset[col])

    features = ['city', 'age', 'calls', 'activeCalls', 'sms', 'data', 'subscription_a', 
                'subscription_b', 'subscription_c', 'subscription_d']

    X = dataset.loc[:, features].values
    y = dataset.loc[:, ['plan']].values

    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)
    centers = kmeans.cluster_centers_
    
    return centers

def create_array(result):
    value = []
    if result['city'] != '':
        value.append(2)
    else:
        value.append(2)
    
    if result['age'] != '':
        value.append(result['age'])
    else:
        value.append(20)
    
    if result['calls'] != '':
        value.append(result['calls'])
    else:
        value.append(11)
    
    if result['activecalls'] != '':
        value.append(result['activecalls'])
    else:
        value.append(19)
    
    if result['sms'] != '':
        value.append(result['sms'])
    else:
        value.append(9)
        
    if result['data'] != '':
        value.append(result['data'])
    else:
        value.append(500)
        
    if 'subscription_a' not in result:
        value.append(0)
    else:
        value.append(1)
    
    if 'subscription_b' not in result:
        value.append(0)
    else:
        value.append(1)
    
    if 'subscription_c' not in result:
        value.append(0)
    else:
        value.append(1)
    
    if 'subscription_d' not in result:
        value.append(0)
    else:
        value.append(1)
    
    return value

def process_center_value(centers):
    c = {}
    counter = 0
    for value in centers:
        count = 0
        for v in value:
            value[count] = float("{:.2f}".format(v))
            count = count + 1
            
        c[counter] = value
        counter = counter + 1
    
    
    return c

def process_sub(centers):
    print(type(centers))
    for key, value in centers.items():
        for i in range(6, 10):
            if value[i] >= 0.45:
                centers[key][i] = 1
            else:
                centers[key][i] = 0
        
        for i in range(0, 6):
            centers[key][i] = round(value[i])
    return centers

def process_center_value_suggestion(pred_center, center_values):
    c = {}
    for value in center_values:
        count = 0
        for v in value:
            if count in [1, 2, 3, 4, 5]:
                value[count] = int(v)
            else:
                value[count] = float("{:.2f}".format(v))
            count = count + 1
            
        c[pred_center] = value
    return c


app = Flask(__name__)

@app.route('/')
def home():
    return render_template("home.html")

@app.route('/about')
def about():
    return render_template("about.html")

@app.route('/result',methods = ['POST', 'GET'])
def result():
    if request.method == 'POST':
        form_values = request.form
        result = form_values.to_dict()
        print("Result: ", result)
        value = create_array(result)
        print(value)
        pred_center = kmeans.predict([value])
        print(pred_center)
        center_value = kmeans.cluster_centers_[pred_center]
        print(center_value)
        c = process_center_value_suggestion(pred_center[0], center_value)
        print(c)
        cent = process_sub(c)
    return render_template("result.html", result = cent)

@app.route('/upload')
def upload_file():
    return render_template('about.html')

@app.route('/uploader', methods = ['GET', 'POST'])
def uploader_file():
    if request.method == 'POST':
        f = request.files['file']
        n_clusters = request.form['n_clusters']
        center = file_process(f, n_cluster=int(n_clusters))
        c = process_center_value(center)
        cent = process_sub(c)
        return render_template("output.html", result = cent)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2021 09:20:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 09:20:12] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Apr/2021 09:21:05] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 09:21:05] "GET /static/css/main.css HTTP/1.1" 304 -


Result:  {'city': 'Chennai', 'age': '25', 'calls': '12', 'activecalls': '24', 'sms': '36', 'data': '2048', 'subscription_c': 'on', 'subscription_d': 'on'}
[2, '25', '12', '24', '36', '2048', 0, 0, 1, 1]
[5]
[[3.16095890e+00 3.63257705e+01 1.79627568e+01 2.70398116e+01
  3.17628425e+01 1.80730223e+03 9.97431507e-02 2.54280822e-01
  5.09845890e-01 4.94434932e-01]]
{5: array([3.160e+00, 3.600e+01, 1.700e+01, 2.700e+01, 3.100e+01, 1.807e+03,
       1.000e-01, 2.500e-01, 5.100e-01, 4.900e-01])}
<class 'dict'>


127.0.0.1 - - [22/Apr/2021 09:22:06] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2021 09:23:29] "POST /uploader HTTP/1.1" 200 -


<class 'dict'>
